In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import requests
import random

In [ ]:
opts = webdriver.ChromeOptions()
opts.headless =True

In [ ]:
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
driver.implicitly_wait(10)

In [ ]:
search_url = "https://www.proquest.com/washingtonpost"
driver.get(search_url)


In [ ]:
driver.find_element(By.XPATH, '//*[@id="product-level-nav"]/li[2]').click()

In [ ]:
keys = """((federal reserve) OR (fed)) AND((money supply) OR (open market operations) OR
(quantitative easing) OR (monetary policy) OR (fed funds rate) OR (overnight lending rate) OR
Bernanke OR Volker OR Greenspan OR Yellen OR Powell OR (central bank) OR (interest rates) OR (fed chairman) OR
(fed chair) OR (lender of last resort) OR (discount window) OR (European Central Bank) OR ECB OR (Bank of England) OR
(Bank of Japan) OR BOJ OR (Bank of China) OR Bundesbank OR (Bank of France) OR (Bank of Italy))"""

In [ ]:
driver.find_element(By.XPATH,'//*[@id="select_multiDateRange"]/option[10]').click()
# Start
driver.find_element(By.XPATH,'//*[@id="month2"]/option[2]').click()
driver.find_element(By.XPATH,'//*[@id="day2"]/option[2]').click()
driver.find_element(By.XPATH,'//*[@id="year2"]').send_keys('2000')
# End
driver.find_element(By.XPATH,'//*[@id="month2_0"]/option[13]').click()
driver.find_element(By.XPATH,'//*[@id="day2_0"]/option[32]').click()
driver.find_element(By.XPATH,'//*[@id="year2_0"]').send_keys('2021')

In [ ]:
driver.find_element(By.XPATH,'//*[@id="SourceType_Newspapers"]').click()
driver.find_element(By.XPATH,'//*[@id="Language_ENG"]').click()

In [ ]:
driver.find_element(By.XPATH, '//*[@id="queryTermField"]').send_keys(keys)

In [ ]:
driver.find_element(By.XPATH, '//*[@id="sortType"]/option[2]').click() # oldest first

In [ ]:
#page_num = 1 # go to the page you want
#driver.find_element(By.XPATH, '//*[@id="pageNbrField"]').clear()
#driver.find_element(By.XPATH, '//*[@id="pageNbrField"]').send_keys(page_num)
#driver.find_element(By.XPATH, '//*[@title="Jump to page"]').click()


In [ ]:
driver_temp = webdriver.Chrome(options = opts)
driver_temp.implicitly_wait(10)

In [ ]:
dataset = []
pattern_date = '[0-9]{1,2}\s[A-Za-z]{3,5}\s[0-9]{4}'

In [ ]:
num_pages_visited = 0

while True:
    
    articles = driver.find_elements(By.CLASS_NAME, 'resultItem')
    num_articles_scraped = 0 # per page
    driver_temp.get(driver.current_url)
    for article in articles:

        title = article.find_element(By.XPATH, './/*[@id="citationDocTitleLink"]').get_attribute("title")
        date_text = article.find_element(By.CLASS_NAME,'newspaperArticle').text
        date = re.search(pattern_date, date_text).group()
    
        link = article.find_element(By.XPATH, './/*[@id="citationDocTitleLink"]').get_attribute("href")
        

        driver_temp.execute_script("window.open('');")
        driver_temp.switch_to.window(driver_temp.window_handles[1])
        driver_temp.get(link)
        
        try:
            WebDriverWait(driver_temp, 5).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="fullTextZone"]/text'))
            )
            full_text = driver_temp.find_element(By.XPATH, '//*[@id="fullTextZone"]/text').text
        except:
            full_text = np.nan
        
        try:
            abstract = driver_temp.find_element(By.CLASS_NAME,'abstract').text
        except NoSuchElementException:
            abstract = np.nan
        
    
        article_info = {'date': date, 'title': title, "abstract": abstract, 'full_text': full_text}
        dataset.append(article_info)
        
        num_articles_scraped += 1
        print(f"Number of scraped articles per page: {num_articles_scraped}")
        driver_temp.close()
        driver_temp.switch_to.window(driver_temp.window_handles[0])
        time.sleep(random.uniform(5, 10))
    
    print("\n")
    try:
        num_pages_visited += 1
        print(f"Number of scraped pages: {num_pages_visited}\n")
        driver.find_element(By.CSS_SELECTOR, "[title^='Next Page']").click()
        time.sleep(random.uniform(5,10))
    except NoSuchElementException:
        break
        
driver_temp.quit()
driver.quit()
df = pd.DataFrame(dataset)

In [ ]:
df

In [ ]:
df.to_csv("news_data.csv", index = False)